In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [5]:
from langchain.tools import tool
import base64
from langchain_openai import ChatOpenAI
from langchain.schema.messages import HumanMessage

from pydantic import BaseModel, Field
from typing import Type, Optional

In [8]:
class ImageInput(BaseModel):
    path: str = Field(description="path of an image")


@tool
def get_keywords_from_gpt(path: str, ) -> str:
    """Returns a solution dict with keys keyword and solution in it. It analyzes the image of the math\
        problem and provides the soltution of the math problem and a keyword that can be utilized to search the youtube for solving the problem"""
    
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
        

    chat = ChatOpenAI(model='gpt-4-vision-preview', max_tokens=256)
    base64_image = encode_image(path)

    output = chat.invoke([
        HumanMessage(
            content=[
                {"type": "text", "text": "What is this image about? Focus on detials and relpy in the json format given:\n 'keyword': 'keyword to search for the related videos here', 'solution': 'solution to the question in the video here' "},
                {"type": "image_url", 
                "image_url": {
                    "url": f"data:image/png;base64," + base64_image,
                    "detail": "auto"
                    }}
                ])
    ])
    
    return output.content
    
# get_keywords_from_gpt.invoke("linear-equations-for-sat-55-1.png")

In [70]:
import os
from googleapiclient.discovery import build

@tool
def get_yt_videos(keyword: str) -> list:
    """Returns a list of url of youtube videos retrieved using the youtube api. This tool is helpful in creating the undertading for the user and you must call it each time at the end to generate urls for the keywrods"""
    api_key = os.getenv("YOUTUBE_API_KEY")

    # Initialize the YouTube resource object
    youtube = build("youtube", "v3", developerKey=api_key)

    # Search for videos related to the given topic
    request = youtube.search().list(
        q=keyword,
        part="snippet",
        type="video",
        maxResults=3
    )

    response = request.execute()

    # Extract video URLs from the response
    video_urls = []
    for item in response["items"]:
        video_id = item["id"]["videoId"]
        video_url = f"https://www.youtube.com/watch?v={video_id}"
        video_urls.append(video_url)

    return video_urls

In [71]:
tools = [get_keywords_from_gpt, get_yt_videos]

In [72]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [81]:
from langchain.prompts import MessagesPlaceholder

functions = [format_tool_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant. Your task is to help students understand the mathematics concepts deeply. Instructions that must be followed:\
- You have access to multiple tools and for every query if it relates to a maths problem, always call the `get_yt_videos` tool.\
- If user input is a file path, then call the `get_keywords_from_gpt` tool and use its output to call the `get_yt_videos` tool.\
- Always combine the outputs from the tools and then return them in a structrued format."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [82]:
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [84]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [85]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [86]:
agent_executor.invoke({"input": "I am stuck in an algebra question. Give me generic video urls for algebra by using `get_yt_videos` tool"})



> Entering new AgentExecutor chain...

Invoking: `get_yt_videos` with `{'keyword': 'algebra'}`


['https://www.youtube.com/watch?v=NybHckSEQBI', 'https://www.youtube.com/watch?v=AoUnw7jcHhg', 'https://www.youtube.com/watch?v=LDIiYKYvvdA']Here are some generic video URLs for algebra that you can refer to:

1. [Algebra Basics: What Is Algebra? - Math Antics](https://www.youtube.com/watch?v=NybHckSEQBI)
2. [Algebra Introduction - Basic Overview - Online Crash Course Review Video Tutorial Lessons](https://www.youtube.com/watch?v=AoUnw7jcHhg)
3. [Algebra - Solving Equations - Math Antics](https://www.youtube.com/watch?v=LDIiYKYvvdA)

These videos will help you understand algebra concepts better.

> Finished chain.


{'input': 'I am stuck in an algebra question. Give me generic video urls for algebra by using `get_yt_videos` tool',
 'chat_history': [HumanMessage(content='I am stuck in an algebra question. Give me generic video urls for algebra by using `get_yt_videos` tool'),
  AIMessage(content='Here are some generic video URLs for algebra that you can refer to:\n\n1. [Algebra Basics: What Is Algebra? - Math Antics](https://www.youtube.com/watch?v=NybHckSEQBI)\n2. [Algebra Introduction - Basic Overview - Online Crash Course Review Video Tutorial Lessons](https://www.youtube.com/watch?v=AoUnw7jcHhg)\n3. [Algebra - Solving Equations - Math Antics](https://www.youtube.com/watch?v=LDIiYKYvvdA)\n\nThese videos will help you understand algebra concepts better.')],
 'output': 'Here are some generic video URLs for algebra that you can refer to:\n\n1. [Algebra Basics: What Is Algebra? - Math Antics](https://www.youtube.com/watch?v=NybHckSEQBI)\n2. [Algebra Introduction - Basic Overview - Online Crash Cours

In [87]:
agent_executor.invoke({"input": "linear-equations-for-sat-55-1.png"})



> Entering new AgentExecutor chain...

Invoking: `get_keywords_from_gpt` with `{'path': 'linear-equations-for-sat-55-1.png'}`


```json
{
  'keyword': 'algebraic equations',
  'solution': '16'
}
```
Invoking: `get_yt_videos` with `{'keyword': 'algebraic equations'}`


['https://www.youtube.com/watch?v=LDIiYKYvvdA', 'https://www.youtube.com/watch?v=9FuR91H8EVU', 'https://www.youtube.com/watch?v=Z-ZkmpQBIFo']I have found some helpful videos related to algebraic equations for you to refer to:

1. [Algebra - Solving Equations - Math Antics](https://www.youtube.com/watch?v=LDIiYKYvvdA)
2. [Solving Linear Equations - Basic Algebra Shortcut Tricks!](https://www.youtube.com/watch?v=9FuR91H8EVU)
3. [Solving Linear Equations - Algebra Basics](https://www.youtube.com/watch?v=Z-ZkmpQBIFo)

These videos will assist you in understanding and solving algebraic equations effectively.

> Finished chain.


{'input': 'linear-equations-for-sat-55-1.png',
 'chat_history': [HumanMessage(content='I am stuck in an algebra question. Give me generic video urls for algebra by using `get_yt_videos` tool'),
  AIMessage(content='Here are some generic video URLs for algebra that you can refer to:\n\n1. [Algebra Basics: What Is Algebra? - Math Antics](https://www.youtube.com/watch?v=NybHckSEQBI)\n2. [Algebra Introduction - Basic Overview - Online Crash Course Review Video Tutorial Lessons](https://www.youtube.com/watch?v=AoUnw7jcHhg)\n3. [Algebra - Solving Equations - Math Antics](https://www.youtube.com/watch?v=LDIiYKYvvdA)\n\nThese videos will help you understand algebra concepts better.'),
  HumanMessage(content='linear-equations-for-sat-55-1.png'),
  AIMessage(content='I have found some helpful videos related to algebraic equations for you to refer to:\n\n1. [Algebra - Solving Equations - Math Antics](https://www.youtube.com/watch?v=LDIiYKYvvdA)\n2. [Solving Linear Equations - Basic Algebra Shortc